### Testing how to Run Ollama locally

Testing dummy runs, prompting with LCEL, function calling and function calling with LCEL

In [64]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser, JsonKeyOutputFunctionsParser
from langchain.chat_models import ChatOllama
from langchain.schema import HumanMessage
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
import requests, json, os
from bs4 import BeautifulSoup
import json
from IPython.display import display, Markdown, Latex

In [50]:
os.environ["OPENAI_API_KEY"] = ""

#### Dummy run with LCEL

Load model. Make sure to have followed instructions [here](https://github.com/jmorganca/ollama) to get started with Ollama

In [12]:
ollama_llm = "llama2"

In [14]:
summary_template = """

Using internal knowledgre, answer in short the following question

> {question}

---
if the question cannot be answered using your internal knowledge, simply summarize the text. Include all factual information, numbers, stats, etc.
"""

summary_prompt = ChatPromptTemplate.from_template(summary_template)


chain = summary_prompt | ChatOllama(model=ollama_llm, temperature=0) | StrOutputParser()

chain.invoke(
    {
        "question":"what is langsmith",
    }
)


'I apologize, but I\'m a large language model, I cannot provide an answer to the question "what is langsmith" as it is not a valid or recognized term in any field of study. It is possible that you may have misspelled the term or it could be a made-up word with no actual meaning. Can you please provide more context or clarify the term you are asking about?'

In [15]:
chain.invoke(
    {
        "question":"what is One Piece?",
    }
)

'One Piece is a popular Japanese manga and anime series created by Eiichiro Oda. The story follows the adventures of Monkey D. Luffy, a young boy who becomes a pirate after eating the Gum-Gum Fruit, which gives him the ability to stretch and manipulate his body like rubber.\n\nHere are some key facts about One Piece:\n\n* The series was first published in 1997 and has been ongoing since then.\n* There are currently over 900 chapters of the manga and 20 seasons of the anime.\n* The series has sold over 430 million copies worldwide, making it one of the best-selling manga series of all time.\n* The anime has been dubbed into many languages and has a global following, with episodes airing in over 80 countries.\n* The series has also spawned various spin-offs, including video games, movies, and merchandise.\n* The main character, Monkey D. Luffy, is known for his unique abilities and his strong sense of justice, which drives him to search for the ultimate treasure known as "One Piece" in o

Great, the above works just as with the OpenAI model tested before [here](https://github.com/jzamalloa1/langchain_learning/blob/main/ra.ipynb)

Now let's try to interact with it using information from the web as a RAG

In [16]:
def scrape_text(url: str):
    # Send a GET request to the webpage
    try:
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the content of the request with BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")

            # Extract all text from the webpage
            page_text = soup.get_text(separator=" ", strip=True)

            # Print the extracted text
            return page_text
        else:
            return f"Failed to retrieve the webpage: Status code {response.status_code}"
    except Exception as e:
        print(e)
        return f"Failed to retrieve the webpage: {e}"

In [19]:
url = "https://blog.langchain.dev/announcing-langsmith/" # example url to play with

page_content = scrape_text(url)

In [21]:
template = """Summarize the following question based on the context:

Question: {question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_template(template)

In [26]:
chain = prompt | ChatOllama(model=ollama_llm, temperature=0) | StrOutputParser()

chain.invoke({
    "question":"what is langsmith",
    "context":page_content
})

'LangSmith is a unified platform for debugging, testing, evaluating, and monitoring LLM applications. It provides a range of features to help developers build robust and reliable LLM-powered applications, including:\n\n1. Debugging: LangSmith gives developers full visibility into model inputs and output of every step in the chain of events, making it easy to experiment with new chains and prompt templates and spot the source of unexpected results, errors, or latency issues.\n2. Testing: LangSmith integrates seamlessly with open-source evaluation modules, including heuristic evaluations and LLM evaluations. It also provides tools for creating datasets from traces and uploading them manually.\n3. Evaluating: LangSmith provides a clear interface for letting developers easily see the inputs and outputs for each data point, which can help build intuition when interacting with LLMs.\n4. Monitoring: LangSmith tracks system-level performance (such as latency and cost) and user feedback to help

Same as before

#### Function calling in native

Let's do a small example of Ollama using function calling

In [28]:
json_schema = {
    "title": "Person",
    "description": "Identifying information about a person.",
    "type": "object",
    "properties": {
        "name": {"title": "Name", "description": "The person's name", "type": "string"},
        "age": {"title": "Age", "description": "The person's age", "type": "integer"},
        "fav_food": {
            "title": "Fav Food",
            "description": "The person's favorite food",
            "type": "string",
        },
    },
    "required": ["name", "age"],
}

In [31]:
message_template = [
    HumanMessage(content="Use the following JSON schema when answering questions about people:"),
    HumanMessage(content=json.dumps(json_schema, indent=2)),
    HumanMessage(content="Tell me about the following person: El Bryan is a guy who lives around the corner. He is 35 and likes Korean food")
]

In [33]:
chat_model = ChatOllama(
    model="llama2",
    format="json",
)

chat_response = chat_model(message_template)

chat_response

AIMessage(content='{\n"name": "El Bryan",\n"age": 35,\n"fav_food": "Korean food"\n}')

In [42]:
json.loads(chat_response.content)

{'name': 'El Bryan', 'age': 35, 'fav_food': 'Korean food'}

#### Function calling with LCEL

In [58]:
template = """Tell some key facts about {person} based on the context:

Context: 
{context}

"""

prompt = ChatPromptTemplate.from_template(template)

function_calls = [
    {
        "name": "Person",
        "description": "Identifying information about a person.",
        "type": "object",
        "properties": 
        {
            "name": {"title": "Name", "description": "The person's name", "type": "string"},
            "age": {"title": "Age", "description": "The person's age", "type": "integer"},
            "fav_food": {
                "title": "Fav Food",
                "description": "The person's favorite food",
                "type": "string",
                },
        },
        "required": ["name", "age"],
    }
]

ollama_model = ChatOllama(model="llama2", temperature=0)

person_chain = prompt | ollama_model.bind(function_call={"name":"Person"}, functions=function_calls)


In [59]:
person_chain.invoke(
    {"person":"Roberto",
     "context":"Damian, who was 37, found that chinese food was best. Roberto is another guy who likes indian food better. Roberto is 41."
    }
)

ValueError: Ollama call failed with status code 400. Details: invalid options: function_call, functions

**Ollama unfortunately cannot take in function calling through LCEL** It can only do it natively. OpenAI however can (below)

In [60]:
openai_llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)

person_chain = prompt | openai_llm.bind(function_call={"name":"Person"}, functions=function_calls)

In [61]:
person_chain.invoke(
    {"person":"Roberto",
     "context":"Damian, who was 37, found that chinese food was best. Roberto is another guy who likes indian food better. Roberto is 41."
    }
)

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "name": "Roberto",\n  "age": 41,\n  "foodPreference": "Indian"\n}', 'name': 'Person'}})

We can go one step further and **parse the JSON LCEL output** (Documentation [here](https://python.langchain.com/docs/expression_language/cookbook/prompt_llm_parser))

In [63]:
person_chain = prompt | openai_llm.bind(function_call={"name":"Person"}, functions=function_calls) | JsonOutputFunctionsParser()

person_chain.invoke(
    {"person":"Roberto",
     "context":"Damian, who was 37, found that chinese food was best. Roberto is another guy who likes indian food better. Roberto is 41."
    }
)

{'name': 'Roberto', 'age': 41, 'foodPreference': 'Indian'}

Or if you want to specific call on a definition within the function call output you can do the below (I wouldn't recommend this since your output loses information, I would rather do this manually afterwards)

In [65]:
person_chain = prompt | openai_llm.bind(function_call={"name":"Person"}, functions=function_calls) | JsonKeyOutputFunctionsParser(key_name="age")

person_chain.invoke(
    {"person":"Roberto",
     "context":"Damian, who was 37, found that chinese food was best. Roberto is another guy who likes indian food better. Roberto is 41."
    }
)

41